In [1]:
#%reset
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
sys.path.append('./modules')

import warnings
#import re

import pandas as pd
#import numpy as np
from sklearn import metrics

## import modules to build pipelines
import pipemodules as pm
import projecthandle as proj
import run_grid as rg

#% matplotlib inline

In [8]:
import pickle
import pipemodules as pm
import pandas as pd


def quality_filter(all_data, min_train_score=0.75, max_diff=0.15):

    results = all_data.results

    results.reset_index(drop=True, inplace=True)

    ## filter results and eliminate poor models
    for i in range(0,len(results)):
        if results.mean_train_score[i] > min_train_score \
        and abs(results.mean_test_score[i] - results.mean_train_score[i]) < max_diff:
            continue
        else: 
            results.drop(i, axis=0, inplace=True)

    results.reset_index(drop=True, inplace=True)

    ## create analysis set
    # set arrays for results
    dev_set_score = []
    eval_set_score = []
    dev_evs = []
    eval_evs = []
    dev_mae = []
    eval_mae = []
    dev_mse = []
    eval_mse = []
    dev_medae = []
    eval_medae = []
    method_ids = []
    parameters = []


    for i in range(0,len(results)):
        ## take method_ids and build estimator for current method
        string = results.method_ids[i] # retrive method id
        setup = eval(string) # convert to iterable array

        temp = pm.search_random_forest() #initiate class

        # set the estimator type and initiate estimator class
        _,clf,_ = temp.set_method(setup[2]) 

        # get the development set features
        X_dev_temp, _ = pm.get_X(dev_set.matrix_raw, \
                                 meth.indvals[setup[0]][setup[1]]) 
        # get the evaluation set features
        X_eval_temp, _ = pm.get_X(eval_set.matrix_raw, \
                                  meth.indvals[setup[0]][setup[1]]) 

        del temp

        # retreive hyper-parameters
        params = results['params'][i]
        # set estimator hyper-parameters
        clf.set_params(**params)

        # fit the estimator to the development set
        clf.fit(X_dev_temp, dev_set.y_raw)
        # predict the evaluation set
        eval_predict = clf.predict(X_eval_temp)
        # predict the development set - for metrics
        dev_predict = clf.predict(X_dev_temp)

        # add calculated metrics, methods, and parameters to lists for results
        dev_set_score.append(clf.score(X_dev_temp, dev_set.y_raw))
        eval_set_score.append(clf.score(X_eval_temp, eval_set.y_raw))
        dev_evs.append(metrics.explained_variance_score(dev_predict, dev_set.y_raw))
        eval_evs.append(metrics.explained_variance_score(eval_predict, eval_set.y_raw))
        dev_mae.append(metrics.mean_absolute_error(dev_predict, dev_set.y_raw))
        eval_mae.append(metrics.mean_absolute_error(eval_predict, eval_set.y_raw))
        dev_mse.append(metrics.mean_squared_error(dev_predict, dev_set.y_raw))
        eval_mse.append(metrics.mean_squared_error(eval_predict, eval_set.y_raw))
        dev_medae.append(metrics.median_absolute_error(dev_predict, dev_set.y_raw))
        eval_medae.append(metrics.median_absolute_error(eval_predict, eval_set.y_raw))
        method_ids.append(string)
        parameters.append(params)

    # create dictionary object from results
    evaluation_results = {'dev_set_score':dev_set_score, 'eval_set_score':eval_set_score, \
                         'method_ids':method_ids, 'parameters':parameters, 'dev_evs':dev_evs, \
                         'eval_evs':eval_evs, 'dev_mae':dev_mae, 'eval_mae':eval_mae, \
                         'dev_mse': dev_mse, 'eval_mse':eval_mse, 'dev_median_ae':dev_medae, \
                         'eval_median_ae':eval_medae}
    

    # re-rank and sort filtered methods by test-score (r**2)
    analysis_set = pd.DataFrame(evaluation_results)
    array = np.array(analysis_set['eval_set_score'])
    temp = array.argsort()[::-1]
    ranks = np.empty(len(array), int)
    ranks[temp] = np.arange(len(array))
    analysis_set['rank_test_score'] = ranks
    analysis_set.sort_values(by='rank_test_score', inplace=True)
    analysis_set.reset_index(drop=True, inplace=True)
    
    return analysis_set

/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=1.094e-02, with an active set of 8 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 20 iterations, i.e. alpha=5.470e-03, with an active set of 14 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=3.718e-03, with an active set of 17 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)

In [54]:
pd.DataFrame.to_dict

In [55]:
to_drop = []
for i in range(0, len(evaluation_results)-1):
    if str(evaluation_results.method_ids[i])==str(evaluation_results.method_ids[i+1]):
        to_drop.append(i+1)
    else:
        continue

for i in to_drop:
    evaluation_results.drop(i, inplace=True)

evaluation_results.sort_values(by='rank_test_score', inplace=True)
evaluation_results.reset_index(drop=True, inplace=True)

In [3]:
X,y,labels = proj.set_input('./input_files/rdkit_descriptors.csv')
results = rg.auto_grid(X, y, labels)
#proj.save_eval('./results_test',results)
#analysis_set = quality_filter(results)

In [ ]:
    currind = 0
    to_drop = []
    for i in range(0, len(evaluation_results)-1):
        if abs(evaluation_results.eval_set_score[currind] - evaluation_results.eval_set_score[i+1]) < 0.01 \
            and abs(evaluation_results.dev_set_score[i] - evaluation_results.dev_set_score[i+1]) < 0.01:
            to_drop.append(i+1)
        else:
            currind = i
    
    for i in to_drop:
        evaluation_results.drop(i, inplace=True)
    
    evaluation_results.sort_values(by='rank_test_score', inplace=True)
    evaluation_results.reset_index(drop=True, inplace=True)
    
    to_drop = []
    for i in range(0, len(evaluation_results)-1):
        if str(evaluation_results.method_ids[i])==str(evaluation_results.method_ids[i+1]):
            to_drop.append(i+1)
        else:
            continue

    for i in to_drop:
        evaluation_results.drop(i, inplace=True)

    evaluation_results.sort_values(by='rank_test_score', inplace=True)
    evaluation_results.reset_index(drop=True, inplace=True)

In [4]:
X.to_dict()

{'AlogP': {0: 1.01779,
  1: 0.32000000000000001,
  2: 3.2115999999999998,
  3: 3.2115999999999998,
  4: 3.2115999999999998,
  5: 1.7337,
  6: -0.1071,
  7: 1.9543999999999999,
  8: 0.39119999999999999,
  9: 3.5920000000000001,
  10: 1.3966000000000001,
  11: 1.0270999999999999,
  12: 0.25700000000000001,
  13: 2.3296000000000001,
  14: 3.3224999999999998,
  15: 0.86650000000000005,
  16: 0.96099999999999997,
  17: 0.0055999999999999999,
  18: 1.3503000000000001,
  19: 1.8926000000000001,
  20: 0.53280000000000005,
  21: 1.1702999999999999,
  22: 4.5839999999999996,
  23: 4.8407800000000005,
  24: 3.4718,
  25: 4.1020000000000003,
  26: 3.4788999999999999,
  27: 3.4788999999999999,
  28: 1.081,
  29: 2.8391999999999999,
  30: 3.9930000000000003,
  31: 3.5860000000000003,
  32: 4.0545,
  33: 2.8397999999999999,
  34: 2.2347999999999999,
  35: 2.3405999999999998,
  36: 1.6866000000000001,
  37: 1.1499999999999999,
  38: 0.36980000000000002,
  39: 0.75990000000000002,
  40: 1.0815999999999

In [5]:
X.keys()

Index([u'MinAbsPartialCharge', u'NumRadicalElectrons', u'HeavyAtomMolWt',
       u'MaxAbsEStateIndex', u'MaxAbsPartialCharge', u'MaxEStateIndex',
       u'MinPartialCharge', u'ExactMolWt', u'MolWt', u'NumValenceElectrons',
       ...
       u'fr_thiazole', u'fr_thiocyan', u'fr_thiophene', u'fr_unbrch_alkane',
       u'fr_urea', u'AlogP', u'MLogP', u'XLogP', u'MW', u'Tm'],
      dtype='object', length=200)